In [1]:
import napari
import PIL.Image
import numpy as np
from napari_animation import AnimationWidget

PIL.Image.MAX_IMAGE_PIXELS = 1056323868

viewer = napari.Viewer()
animation_widget = AnimationWidget(viewer)
viewer.window.add_dock_widget(animation_widget, area="right")
viewer.scale_bar.visible = True

napari.run()

/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari_animation/_qt/keyframeslist_widget.py:156: FutureWarning: Themes were changed to use evented model with Pydantic's color type rather than the `rgb(x, y, z)`. The `as_dict=True` option will be changed to `as_dict=False` in 0.4.15
  self.setStyleSheet(template(qss_template, **get_theme(theme_name)))


<OpenFile 'https://download.brainimagelibrary.org/df/75/df75626840c76c15/mouseID_362188-191815/CH1_0.35_100um//191815_00001_CH1.tif'>
Reading <File-like object HTTPFileSystem, https://download.brainimagelibrary.org/df/75/df75626840c76c15/mouseID_362188-191815/CH1_0.35_100um//191815_00001_CH1.tif> 

Reading <File-like object HTTPFileSystem, https://download.brainimagelibrary.org/df/75/df75626840c76c15/mouseID_362188-191815/CH1_3.5_100um//191815_00001_CH1_resample.tif> 

Meta: {'name': 'mouseID_362188-191815', 'scale': (100, 0.35, 0.35), 'multiscale': True, 'contrast_limits': [0, 65535]}, data: (109, 28001, 19868)
Meta: {'name': 'ilastik mask', 'scale': (100, 0.35, 0.35), 'multiscale': True}, data: (109, 28001, 19868)
Reading <File-like object HTTPFileSystem, https://download.brainimagelibrary.org/df/75/df75626840c76c15/mouseID_362188-191815/CH1_3.5_100um//191815_00001_CH1_resample.tif> 

Reading <File-like object HTTPFileSystem, https://download.brainimagelibrary.org/df/75/df75626840c76

Traceback (most recent call last):
  File "/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/vispy/app/backends/_qt.py", line 897, in paintGL
    self._vispy_canvas.events.draw(region=None)
  File "/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/_qt/qt_viewer.py", line 982, in on_draw
    layer._update_draw(
  File "/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-pack

In [ ]:
1545339 to 1545335

In [ ]:
1549307 to 1548171

In [3]:
import time
import pickle

start = time.time()
with open("/Users/thomasathey/Documents/mimlab/mouselight/BIL/image_viterbrain_noattr_2.pickle", "rb") as handle:
    viterbi = pickle.load(handle)

end = time.time()
print(end - start)

334.5559539794922


In [12]:
state = viterbi.comp_to_states[1545418][0]
pt1 = viterbi.nxGraph.nodes[state]["point1"]
state = viterbi.comp_to_states[1545432][0]
pt2 = viterbi.nxGraph.nodes[state]["point1"]

In [15]:
print(f"{pt1}, {pt2}")

[   54 17553 13759], [   54 17769 13527]


In [19]:
viterbi.fragment_path = "/Users/thomasathey/Documents/mimlab/mouselight/BIL/image_labels_partial.zarr"
viterbi.shortest_path(pt1, pt2)

[array([   54, 17553, 13759]),
 array([   54, 17565, 13743]),
 array([   54, 17589, 13721]),
 array([   54, 17605, 13696]),
 array([   54, 17607, 13693]),
 array([   54, 17623, 13671]),
 array([   54, 17626, 13632]),
 array([   54, 17693, 13594]),
 array([   54, 17719, 13567]),
 array([   54, 17723, 13558]),
 array([   54, 17724, 13555]),
 array([   54, 17730, 13549]),
 array([   54, 17727, 13532]),
 array([   54, 17745, 13527]),
 array([   54, 17769, 13527]),
 array([   54, 17782, 13510]),
 array([   54, 17769, 13527])]

In [ ]:
import zarr
from skimage import io
from skimage import measure
from tqdm import tqdm
import h5py
import numpy as np

## convert ilastik output to labels in tif format

In [ ]:


f = h5py.File(
    "/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/191815_05401_CH1_Simple Segmentation.h5",
    "r",
)
pred = f.get("exported_data")
pred = pred[:, :, 0]
mask = pred == 1
mask = mask.astype("int32")
labels = measure.label(mask)
io.imsave("/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/54_comps.tif", labels)

## write zarr

In [ ]:
zarr_labels = zarr.zeros((109, 28001, 19868), chunks=(1,1000,1000), dtype='i4')

for z in tqdm(range(52, 57)):
    labels = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/" + str(z) + "_comps.tif")
    zarr_labels[z,:,:] = labels

zarr.save('/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/labels_zarr.zarr', zarr_labels)

In [ ]:
zarr_image = zarr.zeros((109, 28001, 19868), chunks=(1,1000,1000), dtype='i4')

for z in tqdm(range(52, 57)):
    image = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/191815_0" + str(z) + "01_CH1.tif")
    zarr_image[z,:,:] = image

#zarr.save('/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/image_zarr.zarr', zarr_image)

In [ ]:
import zarr
zarr_image = zarr.open("/Users/thomasathey/Documents/mimlab/mouselight/BIL/image_labels_partial.zarr")
zarr_image.shape



In [ ]:
for x1 in range(0, 20000,2000):
    x2 = np.amin([x1+2000, zarr_image.shape[2]])
    im = zarr_image[52:57,:,x1:x2]
    fname = "/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/ilastik/" + str(x1) + ".tif"
    io.imsave(fname, im)

In [ ]:
zarr_probs = zarr.zeros((109, 28001, 19868), chunks=(1,1000,1000), dtype='float')
for x1 in range(0, 20000,2000):
    x2 = np.amin([x1+2000, zarr_image.shape[2]])
    f = h5py.File(
        "/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/ilastik/" + str(x1) + "_Probabilities.h5",
        "r",
    )
    pred = f.get("exported_data")
    probs = pred[:,:,:,0]
    zarr_probs[52:57,:,x1:x2] = probs
    break


zarr.save('/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/probs_zarr.zarr', zarr_probs)

# Viterbrain

In [ ]:
from brainlit.algorithms.generate_fragments.state_generation import state_generation

In [ ]:
sg = state_generation(
    image_path="/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/image_zarr.zarr",
    ilastik_program_path="/Applications/ilastik-1.4.0b21-OSX.app/Contents/ilastik-release/run_ilastik.sh",
    ilastik_project_path="/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/ilastik/bil_slice.ilp",
    chunk_size=[5,1000,1000],
    soma_coords=[],
    parallel=8,
    prob_path = "/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/probs_zarr.zarr",
)

In [ ]:
sg.compute_frags()

## Check labels progress

In [ ]:
from skimage import io
import napari
import PIL.Image
import numpy as np


im = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/test_synaptomes/image.tif")
im_probs = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/test_synaptomes/image_probs.tif")
im_labels = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/test_synaptomes/image_labels.tif")

print("Displaying")
v = napari.Viewer()
v.add_image(im, name="image", contrast_limits=[0, 2**16])
v.add_image(im_probs, name="probs")
v.add_labels(im_probs>0.25, name="0.15")
# v.add_labels(im_probs>0.25, name="0.25")
# v.add_labels(im_probs>0.5, name="0.5")
#v.add_labels(im_labels, name="labels")

In [ ]:
import napari
import PIL.Image
import numpy as np
from napari_animation import AnimationWidget
import pickle
from tqdm import tqdm

In [ ]:
import sys

a = np.zeros((10,10))

sys.getsizeof(a)

In [ ]:
sys.getsizeof(viterbi)

In [ ]:
sys.getsizeof(viterbi.nxGraph)